In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider=IBMQ.get_provider(hub="ibm-q", group="open", project="main")
backend=provider.get_backend("ibmq_armonk")

In [ ]:
qubit_n=0
mem_slot=0

In [ ]:
drive_sigma_us=0.075
freq_span_MHz=30
freq_step_MHz=0.5
test_drive_ampl=0.05

In [ ]:
from experiments.qubit_freq_sweep import qubit_freq_sweep
from experiments.rabi_experiment import rabi_experiment
from experiments.ramsey_experiment import ramsey_experiment
from experiments.test_funx import test_funx

In [ ]:
qubit_freq=qubit_freq_sweep(backend, qubit_n=qubit_n, mem_slot=mem_slot, freq_span_MHz=freq_span_MHz,\
            freq_step_MHz=freq_step_MHz, drive_sigma_us=drive_sigma_us, drive_ampl=test_drive_ampl, shots_per_freq=1024, wait_time=45)

In [ ]:
qubit_freq

In [ ]:
pi_amplitude=rabi_experiment(backend=backend, rough_q_freq_Hz=qubit_freq, qubit_n=qubit_n, \
                mem_slot=0, rabi_points=50, drive_ampl_min=0, drive_ampl_max=0.75, \
                drive_sigma_us=drive_sigma_us,  shots_per_point=1024)

In [ ]:
pi_amplitude

In [ ]:
mean_gnd, mean_exc=test_funx(backend=backend, pi_ampl=pi_amplitude, rough_q_freq_Hz=qubit_freq, qubit_n=qubit_n, \
                mem_slot=0, drive_sigma_us=drive_sigma_us, shots_per_freq=1024)

In [ ]:
mean_gnd, mean_exc

In [ ]:
precise_q_freq_Hz=ramsey_experiment(backend=backend, pi_ampl=pi_amplitude, rough_q_freq_Hz=qubit_freq, \
                mean_gnd=mean_gnd, mean_exc=mean_exc, qubit_n=qubit_n, mem_slot=0, time_max_us=1.8, \
                time_step_us=0.025, drive_sigma_us=0.075, wait_time=45, num_shots=256)

In [ ]:
precise_q_freq_Hz

In [ ]:
pi_amplitude=rabi_experiment(backend=backend, rough_q_freq_Hz=qubit_freq, qubit_n=qubit_n, \
                mem_slot=0, rabi_points=50, drive_ampl_min=0, drive_ampl_max=0.75, \
                drive_sigma_us=drive_sigma_us,  shots_per_point=1024)

In [ ]:
pi_amplitude

In [ ]:
import numpy as np

In [ ]:
drive_duration_ns=10*2
ns=1.0E-9

In [ ]:
drive_duration_sec=drive_duration_ns*ns

In [ ]:
drive_sigma_ns=2.5*2
drive_sigma_sec=drive_sigma_ns*ns

In [ ]:
Amplitude=pi_amplitude

In [ ]:
from qiskit import pulse

In [ ]:
def x_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

In [ ]:
from qiskit.circuit import Parameter

In [ ]:
q_s=Parameter('q_s')

In [ ]:
with pulse.build(backend=backend, default_alignment='sequential', name="Drag_Pulse") as drag_pulse_x:
    drive_duration_samples=x_16(pulse.seconds_to_samples(drive_duration_sec))
    drive_sigma_samples=x_16(pulse.seconds_to_samples(drive_sigma_sec))
    drag_pi_by_2_pls=pulse.Drag(drive_duration_samples, Amplitude/2, drive_sigma_samples, q_s, name="QScale Calibration")
    drag_pi_pulse=pulse.Drag(drive_duration_samples, Amplitude, drive_sigma_samples, q_s, name="Qscale Calibration")
    drive_chanl=pulse.drive_channel(qubit_n)
    pulse.set_frequency(precise_q_freq_Hz, drive_chanl)
    
    pulse.play(drag_pi_by_2_pls, drive_chanl)
    
    pulse.play(drag_pi_pulse, drive_chanl)
    
    pulse.measure(qubits=[qubit_n], registers=[pulse.MemorySlot(mem_slot)])

In [ ]:
with pulse.build(backend=backend, default_alignment='sequential', name="Drag_Pulse") as drag_pulse_neg_y:
    drive_duration_samples=x_16(pulse.seconds_to_samples(drive_duration_sec))
    drive_sigma_samples=x_16(pulse.seconds_to_samples(drive_sigma_sec))
    drag_pi_by_2_pls=pulse.Drag(drive_duration_samples, Amplitude/2, drive_sigma_samples, q_s, name="QScale Calibration")
    drag_pi_pulse=pulse.Drag(drive_duration_samples, Amplitude, drive_sigma_samples, q_s, name="Qscale Calibration")
    drive_chanl=pulse.drive_channel(qubit_n)
    pulse.set_frequency(precise_q_freq_Hz, drive_chanl)
    
    pulse.play(drag_pi_by_2_pls, drive_chanl)
    
    pulse.shift_phase(-np.pi/2, drive_chanl)
    
    pulse.play(drag_pi_pulse, drive_chanl)    
    
    pulse.measure(qubits=[qubit_n], registers=[pulse.MemorySlot(mem_slot)])

In [ ]:
with pulse.build(backend=backend, default_alignment='sequential', name="Drag_Pulse") as drag_pulse_y:
    drive_duration_samples=x_16(pulse.seconds_to_samples(drive_duration_sec))
    drive_sigma_samples=x_16(pulse.seconds_to_samples(drive_sigma_sec))
    drag_pi_by_2_pls=pulse.Drag(drive_duration_samples, Amplitude/2, drive_sigma_samples, q_s, name="QScale Calibration")
    drag_pi_pulse=pulse.Drag(drive_duration_samples, Amplitude, drive_sigma_samples, q_s, name="Qscale Calibration")
    drive_chanl=pulse.drive_channel(qubit_n)
    pulse.set_frequency(precise_q_freq_Hz, drive_chanl)
    
    pulse.play(drag_pi_by_2_pls, drive_chanl)
    
    pulse.shift_phase(+np.pi/2, drive_chanl)
    
    pulse.play(drag_pi_pulse, drive_chanl)    
    
    pulse.measure(qubits=[qubit_n], registers=[pulse.MemorySlot(mem_slot)])

In [ ]:
num_of_experiments=60
q_s_range=np.linspace(-1.5,1.5,num_of_experiments)

In [ ]:
schedules_x=[drag_pulse_x.assign_parameters({q_s: qscale}, inplace=False) for qscale in q_s_range]
schedules_neg_y=[drag_pulse_neg_y.assign_parameters({q_s: qscale}, inplace=False) for qscale in q_s_range]
schedules_y=[drag_pulse_y.assign_parameters({q_s: qscale}, inplace=False) for qscale in q_s_range]

In [ ]:
schedules_x[-1].draw(backend=backend);

In [ ]:
schedules_neg_y[-1].draw(backend=backend);

In [ ]:
schedules_y[-1].draw(backend=backend);

In [ ]:
num_of_shots_per_point=1024

In [ ]:
job_x=backend.run(schedules_x, meas_level=1, meas_return='single', shots=num_of_shots_per_point)

In [ ]:
job_x.wait_for_final_state(timeout=None, wait=30, callback=None)

In [ ]:
result_shed_x=job_x.result(timeout=120)

In [ ]:
import math

def classify(point: complex):
    """Classify the given state as |0> or |1>."""
    def distance(a, b):
        return math.sqrt((np.real(a) - np.real(b))**2 + (np.imag(a) - np.imag(b))**2)
    return int(distance(point, mean_exc) < distance(point, mean_gnd))

In [ ]:
scale_fact=1.0E-14

In [ ]:
_x_val=[]
for i in range(len(q_s_range)):
    iq_data=result_shed_x.get_memory(i)[:,qubit_n]*scale_fact
    _x_val.append(sum(map(classify, iq_data))/num_of_shots_per_point)

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.scatter(q_s_range, _x_val, color='black');

In [ ]:
job_neg_y=backend.run(schedules_neg_y, meas_level=1, meas_return='single', shots=num_of_shots_per_point)

In [ ]:
job_neg_y.wait_for_final_state(timeout=None, wait=30, callback=None)

In [ ]:
result_shed_neg_y=job_neg_y.result(timeout=120)

In [ ]:
neg_y_val=[]
for i in range(len(q_s_range)):
    iq_data=result_shed_neg_y.get_memory(i)[:,qubit_n]*scale_fact
    neg_y_val.append(sum(map(classify, iq_data))/num_of_shots_per_point)

In [ ]:
pyplot.scatter(q_s_range, neg_y_val, color="blue")

In [ ]:
job_y=backend.run(schedules_y, meas_level=1, meas_return='single', shots=num_of_shots_per_point)

In [ ]:
job_y.wait_for_final_state()

In [ ]:
result_shed_y=job_y.result(timeout=120)

In [ ]:
_y_val=[]
for i in range(len(q_s_rang)):
    iq_data=result_shed_y.get_memory(i)[:, qubit_n]*scale_fact
    _y_val.append(sum(map(classify, iq_data))/num_of_shots_per_point)

In [ ]:
pyplot.scatter(q_s_range,_y_val, color="red")

In [ ]:
with pulse.build(backend=backend, default_alignment='sequential', name="Drag_Pulse") as drag_pulse_neg_y:
    drive_duration_samples=x_16(pulse.seconds_to_samples(drive_duration_sec))
    drive_sigma_samples=x_16(pulse.seconds_to_samples(drive_sigma_sec))
    drag_pls=pulse.Drag(drive_duration_samples, Amplitude, drive_sigma_samples, q_s, name="QScale Calibration")
    drive_chanl=pulse.drive_channel(qubit_n)
    pulse.set_frequency(precise_q_freq_Hz, drive_chanl)
    pulse.shift_phase(-np.pi/2, drive_chanl)
    pulse.play(drag_pls, drive_chanl)
    pulse.measure(qubits=[qubit_n], registers=[pulse.MemorySlot(mem_slot)])

with pulse.build(backend=backend, default_alignment='sequential', name="Drag_Pulse") as drag_pulse_y:
    drive_duration_samples=x_16(pulse.seconds_to_samples(drive_duration_sec))
    drive_sigma_samples=x_16(pulse.seconds_to_samples(drive_sigma_sec))
    drag_pls=pulse.Drag(drive_duration_samples, Amplitude, drive_sigma_samples, q_s, name="QScale Calibration")
    drive_chanl=pulse.drive_channel(qubit_n)
    pulse.shift_phase(+np.pi/2, drive_chanl)
    pulse.play(drag_pls, drive_chanl)
    pulse.measure(qubits=[qubit_n], registers=[pulse.MemorySlot(mem_slot)])

with pulse.build(backend=backend, default_alignment='sequential', name="Drag_Pulse") as drag_pulse_x:
    drive_duration_samples=x_16(pulse.seconds_to_samples(drive_duration_sec))
    drive_sigma_samples=x_16(pulse.seconds_to_samples(drive_sigma_sec))
    drag_pls=pulse.Drag(drive_duration_samples, Amplitude, drive_sigma_samples, q_s, name="QScale Calibration")
    drive_chanl=pulse.drive_channel(qubit_n)
    pulse.set_frequency(precise_q_freq_Hz, drive_chanl)
    pulse.play(drag_pls, drive_chanl)
    pulse.measure(qubits=[qubit_n], registers=[pulse.MemorySlot(mem_slot)])